In [3]:
# define rooms and items
# master's chambers
sarcophagus = {
    "name": "sarcophagus",
    "type": "furniture",
}

altar = {
    "name": "altar",
    "type": "furniture",
}

door_a = {
    "name": "door a",
    "type": "door",
}

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

masters_chambers = {
    "name": "master's chambers",
    "type": "room",
}

# throne room
throne = {
    "name": "throne",
    "type": "furniture",
}

pile_of_gold = {
    "name": "pile of gold",
    "type": "death",
}

cobra = {
    "name": "A COBRA and it bites you. YOU ARE DEAD",
    "type": "death",
}

door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}
key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

throne_room = {
    "name": "throne room",
    "type": "room",
}

# pharaoh's room
pharaohs_bed = {
    "name": "pharaoh's bed",
    "type": "furniture",
}

dresser = {
    "name": "dresser",
    "type": "furniture",
}

door_b = {
    "name": "door b",
    "type": "door",
}

key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}

door_d = {
    "name": "door d",
    "type": "door",
}

key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}

room_of_pharaoh = {
    "name": "pharaoh's room",
    "type": "room",
}
# yard
sphinx = {
    "name": "sphinx",
    "type": "death",
}

scorpion = {
    "name": "A HUGE SCORPION",
    "type": "death",
}

yard = {
    "name": "yard",
    "type": "room",
}
# GLOBAL
outside = {
  "name": "outside"
}

all_rooms = [masters_chambers, throne_room, room_of_pharaoh, yard, outside]

all_doors = [door_a, door_b, door_c, door_d]

# define which items/rooms are related

object_relations = {
    #master's chambers
    "master's chambers": [sarcophagus, altar, door_a],
    "altar": [key_a],
    "door a": [masters_chambers, throne_room],
    
    #throne_room 
    "throne room": [throne, sphinx, door_b, door_c],
    "throne": [key_b],
    "sphinx": [scorpion], 
    "door b": [throne_room, room_of_pharaoh],
    "door c": [throne_room, yard],
    
    #room_of_pharaoh
    "pharaoh's room": [pharaohs_bed, dresser, door_b],
    "pharaoh's bed": [key_c],
    "dresser": [key_d],
    
    #living_room y sus muebles
    "yard": [pile_of_gold, door_d],
    "pile of gold": [cobra],
    "door d": [yard, outside],
    
    #salida 
    "outside": [door_d],
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": masters_chambers,
    "keys_collected": [],
    "target_room": outside
}

def linebreak():
    """
    Print a line break
    """
    print("\n\n")
    
def reset_game():
    """
    Reset the game state to the initial state.
    """
    game_state["current_room"] = INIT_GAME_STATE["current_room"]
    game_state["keys_collected"] = INIT_GAME_STATE["keys_collected"]
    game_state["target_room"] = INIT_GAME_STATE["target_room"]
    
    # Restaurar la llave key_a en el altar después de reiniciar el juego
    object_relations["altar"] = [key_a]
    object_relations["throne"] = [key_b]
    object_relations["pharaoh's bed"] = [key_c]
    object_relations["dresser"] = [key_d]

def start_game():
    """
    Start the game
    """
    print("You wake up on a sarcophagus and find yourself inside a big room where the walls are full of hieroglyphs soon you realize you are trap inside an egiptian temple a place with no windows which you have never been to before. The only thing you can remember is that last night you were celebrating your birthday's party. You feel lost and confuse and the only thing you want to do is to find the way out!")
    play_room(game_state["current_room"])
                        
def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You've just escaped THE TEMPLE OF DOOM!")
    else:
        print("You are now in " + room["name"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine'?").strip()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine?").strip())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    
    for item in object_relations[current_room["name"]]:
        if item["name"] == item_name:
            output = "You examine " + item_name + ". "
            if item["type"] == "door":
                have_key = False
                # Verificar si tienes alguna llave que pueda abrir esta puerta
                for key in game_state["keys_collected"]:
                    # Comprobar si el objeto key tiene un atributo "target" y si coincide con la puerta actual
                    if "target" in key and key["target"] == item:
                        have_key = True
                        break  # Salir del bucle tan pronto como se encuentre la llave correcta
                
                if have_key:
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)              
                else:
                    output += "It is locked but you don't have the key."
            else:
                if item["name"] in object_relations and len(object_relations[item["name"]]) > 0:
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                    
                    # Verificar si el objeto encontrado es una "death" (muerte)
                    if item["type"] == "death":
                        if item["name"] == "pile of gold":
                            print("You find a beautiful COBRA. GAME OVER.")
                            reset_game()  # Reiniciar el juego al estado inicial
                            play_room(game_state["current_room"])  # Volver a jugar desde la habitación inicial
                            return
                        elif item["name"] == "sphinx":
                            print("You find a nice SCORPION. GAME OVER.")
                            reset_game()  # Reiniciar el juego al estado inicial
                            play_room(game_state["current_room"])  # Volver a jugar desde la habitación inicial
                            return
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break
    
    if output is None:
        print("The item you requested is not found in the current room.")
    
    if next_room and input("Do you want to go to the next room? Enter 'yes' or 'no'").strip() == 'yes':
        play_room(next_room)
    else:
        play_room(current_room)

In [ ]:
game_state = INIT_GAME_STATE.copy()

start_game()

You wake up on a sarcophagus and find yourself inside a big room where the walls are full of hieroglyphs soon you realize you are trap inside an egiptian temple a place with no windows which you have never been to before. The only thing you can remember is that last night you were celebrating your birthday's party. You feel lost and confuse and the only thing you want to do is to find the way out!
You are now in master's chambers


What would you like to do? Type 'explore' or 'examine'? explore


You explore the room. This is master's chambers. You find sarcophagus, altar, door a
You are now in master's chambers


What would you like to do? Type 'explore' or 'examine'? examine
What would you like to examine? altar


You examine altar. You find key for door a.
You are now in master's chambers


...

Do you want to go to the next room? Enter 'yes' or 'no' yes


You are now in yard


What would you like to do? Type 'explore' or 'examine'? explore


You explore the room. This is yard. You find pile of gold, door d
You are now in yard


What would you like to do? Type 'explore' or 'examine'? examine


## Presentación del proyecto
https://docs.google.com/presentation/d/1I8WLykVHGOHHbEISw46iwZC6tDVk_rE_fTXnLrGsbzo/edit?usp=sharing